In [2]:
import os
from datasets import load_dataset, concatenate_datasets, DownloadMode
import re
import datetime
from concurrent.futures import ThreadPoolExecutor
import tqdm


/home/senya/.cache/pypoetry/virtualenvs/drip-search-J_7n9XPv-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
ROOT_DIR = '/mnt/e/code/politech/nlp/drip-search'
data_dir = os.path.join(ROOT_DIR, 'data')
subsets = ["20231101.uk", "20231101.en"]
ds = concatenate_datasets([load_dataset("wikimedia/wikipedia", subset, cache_dir=data_dir, download_mode=DownloadMode.REUSE_DATASET_IF_EXISTS)['train'] for subset in subsets ])

In [4]:
drip_titles = ['cp company', 'stone island', 'nike', 'arcteryx', 'adidas', 'timberland', 'chrome hearts']

In [ ]:
ds.filter(lambda x: x['title'].lower() in drip_titles)

Filter:   0%|          | 0/7702534 [00:00<?, ? examples/s]

Filter:  29%|██▉       | 2250000/7702534 [09:54<17:14, 5268.56 examples/s]  

In [ ]:
import tqdm
from concurrent.futures import ThreadPoolExecutor

def check_title_in_drip_titles(doc, drip_titles):
    return doc['title'].lower() in drip_titles

# Function to split the dataset into chunks
def chunkify(lst, n):
    avg = len(lst) // n
    chunks = [lst[i:i + avg] for i in range(0, len(lst), avg)]
    if len(chunks) > n:
        return chunks[:n]
    else:
        return chunks + [chunks[-1]] * (n - len(chunks))

# Split the dataset into 4 chunks
chunks = chunkify(ds, 4)

# Use ThreadPoolExecutor to process the chunks in parallel
with ThreadPoolExecutor(max_workers=4) as executor:
    # Parallelize the filter operation and wrap it with tqdm for progress tracking
    results = list(tqdm.tqdm(executor.map(lambda chunk: [doc for doc in tqdm.tqdm(chunk) if check_title_in_drip_titles(doc, drip_titles)], chunks), total=len(chunks)))

# Flatten the results list (because executor.map returns a list of lists)
drip_docs = [doc for sublist in results for doc in sublist]

print([drip_doc['title'] for drip_doc in drip_docs])




In [ ]:
ds[0:3]

{'id': ['13', '584', '585'],
 'url': ['https://uk.wikipedia.org/wiki/%D0%93%D0%B5%D0%BE%D0%B3%D1%80%D0%B0%D1%84%D1%96%D1%8F',
  'https://uk.wikipedia.org/wiki/%D0%90%D1%82%D0%BE%D0%BC',
  'https://uk.wikipedia.org/wiki/%D0%9C%D1%96%D0%BB%D1%8C%D1%8F%D1%80%D0%B4'],
 'title': ['Географія', 'Атом', 'Мільярд'],
 'text': ['Геогра́фія, або заст. земле́пис (від \xa0— опис Землі; де \xa0— Земля і \xa0— писати, описувати)\xa0— система наук, що вивчає географічну оболонку Землі (епігеосферу), її просторову природну і соціально-економічну різноманітність, господарство і населення планети, окремих її регіонів та країн, а також зв\'язки між природним середовищем і діяльністю людини. В сучасному розумінні поняття «географія» заміщено поняттям «географічні науки».\n\nПоняття \nУперше термін «географія» був запропонований давньогрецьким географом Ератосфеном (276—194 до н.\xa0е.).\n\nСтруктура \n\nОсновне завдання сучасної географії полягає у вивченні законів і закономірностей розміщення та взаємодії 

In [ ]:
for i, doc in enumerate(drip_docs):
    filename = f'{i}-{doc['title'].lower().replace('/', '')}-{datetime.datetime.now().isoformat()}.txt'
    filepath = os.path.join(data_dir, filename) 
    with open(filepath, 'x') as f:
        f.write(doc['text'])